In [1]:
from aws_utils import read_csv_as_dataframe
import pandas as pd
import s3fs
pd.set_option('display.max_colwidth', -1)

ModuleNotFoundError: No module named 'aws_utils'

In [2]:
website="olx"
location="warszawa"
#Metadata
BUCKET = f's3://data-apartments/production/{website}/{location}/'
df_olx = read_csv_as_dataframe(bucket_path=BUCKET, filename=f'{website}_db.csv', timestamp_column_name='downloaded')
df_olx['website'] = 'olx'

NameError: name 'read_csv_as_dataframe' is not defined

In [ ]:
website="gumtree"
location="warszawa"
#Metadata
BUCKET = f's3://data-apartments/production/{website}/{location}/'
df_gumtree = read_csv_as_dataframe(bucket_path=BUCKET, filename=f'{website}_db.csv', timestamp_column_name='downloaded')
df_gumtree['website'] = 'gumtree'

In [ ]:
website="otodom"
location="warszawa"
#Metadata
BUCKET = f's3://data-apartments/production/{website}/{location}/'
df_otodom = read_csv_as_dataframe(bucket_path=BUCKET, filename=f'{website}_db.csv', timestamp_column_name='downloaded')
df_otodom['website'] = 'otodom'

In [ ]:
df = pd.concat([df_olx, df_otodom, df_gumtree], sort=False)

# Data cleaning

In [ ]:
df = df[df.price_per_m.notnull() & df.price.notnull()].copy()

# Max price per district

In [ ]:
df_distr = df[['district', 'price', 'price_per_m']].groupby(['district']).agg({
               'price':['count', 'mean'], 'price_per_m':['mean']    
            }).reset_index()
df_distr.columns = df_distr.columns.droplevel()
df_distr.columns = ['district', 'count', 'mean_price', 'mean_price_per_m']
df_distr.query('count > 1500', inplace=True)
df_distr.sort_values(by=['mean_price_per_m'], ascending=False, inplace=True)

In [ ]:
trace0 = go.Bar(
    x=df_distr['district'].tolist()[:10],
    y=df_distr['mean_price_per_m'].tolist()[:10],
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
)

data = [trace0]
layout = go.Layout(
    title='Gdzie cena metra kwadratowego jest najwyższa?',
    yaxis = go.layout.YAxis(
        tickformat = 'zł'
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='text-hover-bar')

## Offers

In [ ]:
df_distr = df[['district', 'price', 'price_per_m']].groupby(['district']).agg({
               'price':['count', 'mean'], 'price_per_m':['mean']    
            }).reset_index()
df_distr.columns = df_distr.columns.droplevel()
df_distr.columns = ['district', 'count', 'mean_price', 'mean_price_per_m']
df_distr.query('count > 1500', inplace=True)
df_distr.sort_values(by=['count'], ascending=False, inplace=True)

In [ ]:
trace0 = go.Bar(
    x=df_distr['district'].tolist()[:10],
    y=df_distr['count'].tolist()[:10],
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
)

data = [trace0]
layout = go.Layout(
    title='Gdzie wystawiane jest najwięcej ofert sprzedaży nieruchomości?',
    yaxis = go.layout.YAxis(
        title='Liczba ofert',
        tickformat = 'zł'
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='text-hover-bar-offers')

## Area

In [ ]:
df_distr = df[['district', 'area']].groupby(['district']).agg({
               'area':['count', 'mean']    
            }).reset_index()
df_distr
df_distr.columns = df_distr.columns.droplevel()
df_distr.columns = ['district', 'count', 'mean_area']
df_distr.query('count > 1500', inplace=True)
df_distr.sort_values(by=['mean_area'], ascending=False, inplace=True)

df_distr

In [ ]:
trace0 = go.Bar(
    x=df_distr['district'].tolist()[:10],
    y=df_distr['mean_area'].tolist()[:10],
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
)

data = [trace0]
layout = go.Layout(
    title='Gdzie kupisz największe mieszkania?',
    yaxis = go.layout.YAxis(
        title='Średnia powierzchnia mieszkania',
        tickformat = 'm2'
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='text-hover-bar-area')

## Price map distribution

In [ ]:
df_geo = df[df.longitude.notnull() & df.latitude.notnull()].copy()
df_geo = df_geo.query('longitude < 21.165580 and longitude > 20.872942 and latitude < 52.359453 and latitude > 52.128592')
df_geo = df_geo.query('price_per_m < 50000')
df_geo.longitude = df_geo.longitude.astype(float)
df_geo.latitude = df_geo.latitude.astype(float)

In [ ]:
df_geo.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(df_geo.longitude.tolist(), df_geo.latitude.tolist())
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

prices = df_geo.price_per_m.tolist()
scaler = MinMaxScaler()
prices_scaled = scaler.fit_transform(np.array(prices).reshape(len(prices), 1))

fig = plt.figure(figsize=(20, 20))
ax1 = fig.add_subplot(111, projection='3d')

xpos = df_geo.longitude.tolist()
ypos = df_geo.latitude.tolist()
num_elements = len(xpos)
zpos = np.zeros(num_elements)
dx = [0.001]*num_elements
dy = [0.001]*num_elements
dz = prices_scaled.ravel()

ax1.bar3d(xpos, ypos, zpos, dx, dy, dz, color='#00ceaa')
plt.show()

In [ ]:
min_long, max_long = df_geo.longitude.min(), df_geo.longitude.max()
min_lat, max_lat = df_geo.latitude.min(), df_geo.latitude.max()
long = np.linspace(min_long, max_long, 10).tolist()
lat = np.linspace(min_lat, max_lat, 10).tolist()

In [ ]:
longitudes = [20.88051723,
                 20.911873304444438,
                 20.943229378888876,
                 20.974585453333315,
                 21.005941527777754,
                 21.037297602222196,
                 21.068653676666635,
                 21.100009751111074,
                 21.131365825555513,
                 21.16272189999995]
latitudes = [52.12953870000001,
             52.154670755555564,
             52.179802811111124,
             52.204934866666676,
             52.23006692222223,
             52.25519897777779,
             52.28033103333334,
             52.30546308888889,
             52.33059514444445,
             52.355727200000004]
    
def assign_group(row):
    # Assign group based on latitude and longitude
    # This is hard coded, take care of coordinate boundaries!
    global longitudes
    global latitudes
    group_long, group_lat = None, None
    for ind, longitude in enumerate(longitudes):
        if row.longitude <= longitude:
            group_long = str(longitudes[ind-1])
            break
        else:
            continue
            
    for ind, latitude in enumerate(latitudes):
        if row.latitude <= latitude:
            group_lat = str(latitudes[ind-1])
            break
        else:
            continue
    return group_long + "_" + group_lat

df_geo['geo_group'] = df_geo.apply(assign_group, axis=1)
df_geo[['longitude_plot', 'latitude_plot']] = df_geo.geo_group.str.extract("(?P<longitude_plot>\d+.\d+)_(?P<latitude_plot>.+)", expand=True)
df = df_geo[['price_per_m', 'longitude_plot', 'latitude_plot']].groupby(['longitude_plot', 'latitude_plot']).mean().reset_index()

def round_to_pln(x):
    return str(int(x)) + ' zł'

df['price_per_m_formatted'] = df.price_per_m.apply(round_to_pln)

In [ ]:
df

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

mapbox_access_token = 'pk.eyJ1IjoiZGViMWwiLCJhIjoiY2p2cWdodm1yMDhvZTQ4cDl1eGJpcXNxdSJ9.bPBbIP7QMwD9UaVFwxffFw'


data = [
    go.Scattermapbox(
        lat= df['latitude_plot'].tolist(),
        lon= df['longitude_plot'].tolist(),
        text=df['price_per_m_formatted'].tolist(),
        mode='markers',
        hoverinfo='text',
        marker=dict(
            size=16,
            color = df['price_per_m'].tolist(), #set color equal to a variable
            colorscale='Jet',
            showscale=True
        ),
    )
]

layout = go.Layout(
    width=1000,
    height=1000,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=52.235360,
            lon= 21.009063
        ), 
        pitch=0,
        zoom=10
    ),
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Multiple Mapbox')